In [98]:
from tokenizers import Tokenizer, models, trainers, pre_tokenizers, normalizers
from tokenizers.normalizers import Replace
from tokenizers.pre_tokenizers import Split

tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))


In [99]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents(),Replace("\n","Ċ")]
)
tokenizer.normalizer.normalize_str("Héllò hôw\nare ü?")

'hello howĊare u?'

In [100]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation(),Split("Ċ","isolated")]
)
n=tokenizer.normalizer.normalize_str("To be, or not to be:\n\nThat is \nthe question." )
pre_tokenizer.pre_tokenize_str(n)

[('to', (0, 2)),
 ('be', (3, 5)),
 (',', (5, 6)),
 ('or', (7, 9)),
 ('not', (10, 13)),
 ('to', (14, 16)),
 ('be', (17, 19)),
 (':', (19, 20)),
 ('Ċ', (20, 21)),
 ('Ċ', (21, 22)),
 ('that', (22, 26)),
 ('is', (27, 29)),
 ('Ċ', (30, 31)),
 ('the', (31, 34)),
 ('question', (35, 43)),
 ('.', (43, 44))]

In [101]:
special_tokens = ["[UNK]"]
trainer = trainers.WordPieceTrainer(
    vocab_size=8000,
    special_tokens=special_tokens,
    show_progress=False
)

In [102]:
cls_token_id = tokenizer.token_to_id("[UNK]")
print(cls_token_id)

None 1


In [103]:
files = ["input.txt"]
tokenizer.train(files, trainer)
file_name="wp_tokenizer.json"
tokenizer.save(file_name)

In [104]:
# Load your trained tokenizer
tokenizer = Tokenizer.from_file(file_name)

text = "To be, or not to be:\n\nThat is \nthe question."

# Encode
encoding = tokenizer.encode(text)
print(encoding.tokens)

['to ', '##be, ', '##or no', '##t to ', '##be', '##:Ċ', '##Ċ', '##that is ', '##Ċ', '##the ', '##question', '##.']


In [105]:
from tokenizers import decoders

tokenizer.decoder = decoders.Sequence([decoders.Replace("Ċ","\n"), decoders.WordPiece(prefix="##")])
tokenizer.decode(encoding.ids)

'to be, or not to be:\n\nthat is \nthe question.'